In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import numpy as np
import imutils

In [7]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [8]:
TRAINING_DIR = "./train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "./test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

Found 10013 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [9]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [10]:
history = model.fit(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Epoch 1/10
1002/1002 [==============================] - 751s 749ms/step - loss: 0.6030 - acc: 0.6405 - val_loss: 0.4933 - val_acc: 0.7784
INFO:tensorflow:Assets written to: model2-001.model\assets
Epoch 2/10
1002/1002 [==============================] - 625s 624ms/step - loss: 0.3876 - acc: 0.8163 - val_loss: 1.6601 - val_acc: 0.5258
Epoch 3/10
1002/1002 [==============================] - 616s 615ms/step - loss: 0.3208 - acc: 0.8612 - val_loss: 2.9228 - val_acc: 0.4948
Epoch 4/10
1002/1002 [==============================] - 616s 614ms/step - loss: 0.2893 - acc: 0.8758 - val_loss: 3.6740 - val_acc: 0.5000
Epoch 5/10
1002/1002 [==============================] - 657s 656ms/step - loss: 0.2566 - acc: 0.8930 - val_loss: 3.6627 - val_acc: 0.5000
Epoch 6/10
1002/1002 [==============================] - 626s 625ms/step - loss: 0.2390 - acc: 0.9016 - val_loss: 2.9365 - val_acc: 0.5000
Epoch 7/10
1002/1002 [==============================] - 617s 615ms/step - loss: 0.2129 - acc: 0.9148 - val_loss: 

In [11]:
model.save("test_model.h5")

In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
model=load_model("test_model.h5")

results={0:'mask',1:'without mask'}
GR_dict={0:(0,255,0),1:(0,0,255)}

rect_size = 4
cap = cv2.VideoCapture(0) 


haarcascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [2]:
while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break

cap.release()

cv2.destroyAllWindows()

In [3]:
cap.release() 